# Data processing and data analysis

## At the very beginning we're going to say how we're going to separate the frozen and refrigerated.
To make up for the beginning, do it with a diagram
Let's just say that we counted the existing frozen products and refrigerated, frozen is refrigirated and refrigerated is dairy and egg type and type with any meat. Then train a literal model based on the existing refrigerated as well as frozen, and then put the rows with department as missing and other into the model to determine if he is refrigerated or frozen, and finally cleaned up when all the data is categorized with other, refrigerated, and frozen

## Importing and cleaning up


In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
from datetime import datetime
import math
import warnings
import json

In [11]:
data = pd.read_csv('mma_mart_Final_data.csv')
data.head()

C:\Users\啵啵小狗\AppData\Local\Temp\ipykernel_15568\861759679.py:1: DtypeWarning:

Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.



,order_id,product_id,product_name,aisle_id,aisle,department_id,department,type3
0,1,49302,Bulgarian Yogurt,120,yogurt,16,dairy eggs,Refrigerated
1,1,11109,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,other creams cheeses,16,dairy eggs,Refrigerated
2,1,10246,Organic Celery Hearts,83,fresh vegetables,4,produce,Refrigerated
3,1,49683,Cucumber Kirby,83,fresh vegetables,4,produce,Refrigerated
4,1,43633,Lightly Smoked Sardines in Olive Oil,95,canned meat seafood,15,canned goods,Other


In [12]:
# Remove rows with text values in 'aisle_id' and 'department_id'
data_cleaned = data[
    (data['aisle_id'].apply(lambda x: isinstance(x, str) and x.isnumeric())) &
    (data['department_id'].apply(lambda x: isinstance(x, str) and x.isnumeric()))
]

# Display the shape of the cleaned dataset
data_cleaned.shape

(786406, 8)

## Text vectorization to find substitute 

In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

# Load the dataset
data = data_cleaned

# Remove rows with NaN in 'product_name'
data = data.dropna(subset=['product_name'])

# 1. 文本向量化
# 将产品名称转换为向量形式
vectorizer = TfidfVectorizer(max_features=5000)
product_vectors = vectorizer.fit_transform(data['product_name'].unique())

# 使用K-means进行聚类
num_clusters = 100  # 调整为所需的类别数量
kmeans = KMeans(n_clusters=num_clusters)
product_clusters = kmeans.fit_predict(product_vectors)

# 为每个产品分配一个类别标签
product_labels = dict(zip(data['product_name'].unique(), product_clusters))

# 根据类别标签对产品进行编码
data['product_code'] = data['product_name'].map(product_labels)

# 2. Based on product similarity
# Group products by 'aisle' and 'department'
grouped_products = data.groupby(['aisle', 'department'])['product_name'].unique().reset_index()

# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Set a similarity threshold (products with similarity above this threshold are considered similar)
similarity_threshold = 0.1

# Initialize a dictionary to store the product-to-group mapping
product_to_group = defaultdict(int)

# Initialize a group counter
group_counter = 1

# Loop through each product group based on aisle and department
for idx, row in grouped_products.iterrows():
    # Get the products in the current group
    products = row['product_name']

    # Vectorize the product names in the current group
    tfidf_matrix = vectorizer.fit_transform(products)

    # Compute cosine similarity between product names
    cosine_sim = cosine_similarity(tfidf_matrix)

    # Initialize a set to keep track of products that have already been assigned a group
    assigned_products = set()

    # Find pairs of products that are similar based on the threshold
    for i in range(len(products)):
        if products[i] in assigned_products:
            continue

        # Create a new group for the current product
        current_group = group_counter
        product_to_group[products[i]] = current_group
        assigned_products.add(products[i])

        for j in range(i + 1, len(products)):
            if cosine_sim[i, j] > similarity_threshold:
                product_to_group[products[j]] = current_group
                assigned_products.add(products[j])

        # Increment the group counter
        group_counter += 1

# Add a new column to the original dataset to represent the 'substitute_group'
data['substitute_group'] = data['product_name'].map(product_to_group)

# Save the updated dataset to a new CSV file
data.to_csv('updated_dataset.csv', index=False)


d:\ana\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Users\啵啵小狗\AppData\Local\Temp\ipykernel_15568\3620673930.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\啵啵小狗\AppData\Local\Temp\ipykernel_15568\3620673930.py:78: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [14]:
data_test = pd.read_csv('updated_dataset.csv')

In [15]:
# Look at how many UNIQUE VALUES the substitutes_group has
data_test['substitute_group'].nunique()


4160

In [16]:
#Look at the average number of unique products per substitute_group
data_test.groupby('substitute_group')['product_id'].nunique().mean()


7.958173076923077

In [17]:
#Output data with substitutes_group 1000, procuct_id, and product_name.

print(data_test[data_test['substitute_group'] == 1000].groupby('product_id')['substitute_group'].nunique().reset_index())
data_test[data_test['substitute_group'] == 1000]


   product_id  substitute_group
0        4265                 1
1        4580                 1
2        4866                 1
3       17280                 1
4       24777                 1


,order_id,product_id,product_name,aisle_id,aisle,department_id,department,type3,product_code,substitute_group
41776,4240,4866,Chocolate Covered Biscuit Sticks,61,cookies cakes,19,snacks,Other,14,1000
125825,12760,24777,Original Fudge Sticks,61,cookies cakes,19,snacks,Other,9,1000
150509,21820,4265,Chocolate Covered Gluten Free Wafer Bites,61,cookies cakes,19,snacks,Other,63,1000
156881,22488,4580,Scooby-Doo! Cinnamon Sticks Graham Crackers,61,cookies cakes,19,snacks,Other,44,1000
167890,23574,24777,Original Fudge Sticks,61,cookies cakes,19,snacks,Other,9,1000
174494,24236,4265,Chocolate Covered Gluten Free Wafer Bites,61,cookies cakes,19,snacks,Other,63,1000
257314,32631,4580,Scooby-Doo! Cinnamon Sticks Graham Crackers,61,cookies cakes,19,snacks,Other,44,1000
270310,33957,4580,Scooby-Doo! Cinnamon Sticks Graham Crackers,61,cookies cakes,19,snacks,Other,44,1000
397979,47010,4265,Chocolate Covered Gluten Free Wafer Bites,61,cookies cakes,19,snacks,Other,63,1000
531631,60506,4866,Chocolate Covered Biscuit Sticks,61,cookies cakes,19,snacks,Other,14,1000


## With alternatives brought in, we're going to say that we're doing a data cleansing, because there will be Nan present

In [18]:
data_test_2 = pd.read_csv('updated_dataset.csv')

In [19]:
data_test_2

,order_id,product_id,product_name,aisle_id,aisle,department_id,department,type3,product_code,substitute_group
0,1,49302,Bulgarian Yogurt,120,yogurt,16,dairy eggs,Refrigerated,37,4113
1,1,11109,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,other creams cheeses,16,dairy eggs,Refrigerated,68,2948
2,1,10246,Organic Celery Hearts,83,fresh vegetables,4,produce,Refrigerated,97,1598
3,1,49683,Cucumber Kirby,83,fresh vegetables,4,produce,Refrigerated,80,1599
4,1,43633,Lightly Smoked Sardines in Olive Oil,95,canned meat seafood,15,canned goods,Other,47,635
...,...,...,...,...,...,...,...,...,...,...
786400,99578,38689,Organic Reduced Fat Milk,84,milk,16,dairy eggs,Refrigerated,68,2539
786401,99578,47719,Banana Raspberry & Brown Rice Stage 2,92,baby food formula,18,babies,Other,34,106
786402,99578,17949,Mild Cheddar Cheese Sticks,21,packaged cheese,16,dairy eggs,Refrigerated,77,2969
786403,99578,36216,Lime Italian Sparkling Mineral Water,115,water seltzer sparkling water,7,beverages,Other,36,4076


## Data analysis should be in this position

35,000 products with an order frequency of 10,000 or more (total of 9w orders) - product_id

Perform data analysis on data_test_2

In [20]:
data_test_2 = pd.read_csv('updated_dataset.csv')
data_test_2

,order_id,product_id,product_name,aisle_id,aisle,department_id,department,type3,product_code,substitute_group
0,1,49302,Bulgarian Yogurt,120,yogurt,16,dairy eggs,Refrigerated,37,4113
1,1,11109,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,other creams cheeses,16,dairy eggs,Refrigerated,68,2948
2,1,10246,Organic Celery Hearts,83,fresh vegetables,4,produce,Refrigerated,97,1598
3,1,49683,Cucumber Kirby,83,fresh vegetables,4,produce,Refrigerated,80,1599
4,1,43633,Lightly Smoked Sardines in Olive Oil,95,canned meat seafood,15,canned goods,Other,47,635
...,...,...,...,...,...,...,...,...,...,...
786400,99578,38689,Organic Reduced Fat Milk,84,milk,16,dairy eggs,Refrigerated,68,2539
786401,99578,47719,Banana Raspberry & Brown Rice Stage 2,92,baby food formula,18,babies,Other,34,106
786402,99578,17949,Mild Cheddar Cheese Sticks,21,packaged cheese,16,dairy eggs,Refrigerated,77,2969
786403,99578,36216,Lime Italian Sparkling Mineral Water,115,water seltzer sparkling water,7,beverages,Other,36,4076


## Starting to split the test set and training set.
20% test set, 80% training set, randomized by user, 62348 orders, 629914 pieces of data.
The remaining test set is 15588 orders, 156491 pieces of data.

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
data = data_test_2

# Get unique order_ids
unique_order_ids = data['order_id'].unique()

# Split the unique order_ids into train and test
train_order_ids, test_order_ids = train_test_split(unique_order_ids, test_size=0.2, random_state=42)

# Use these order_ids to split the main dataset into train and test datasets
train_data = data[data['order_id'].isin(train_order_ids)]
test_data = data[data['order_id'].isin(test_order_ids)]

# Save the train and test datasets to CSV files
train_data.to_csv('train_dataset.csv', index=False)
test_data.to_csv('test_dataset.csv', index=False)


# Take the training set and train it

In [22]:
data_train = pd.read_csv('train_dataset.csv')

In [23]:
data_test = pd.read_csv('test_dataset.csv')

In [24]:
# Count the number of orders for each product
product_counts = data_train['product_id'].value_counts()

# Create a dataframe with product_ids and their counts
product_popularity = pd.DataFrame({
    'product_id': product_counts.index,
    'order_count': product_counts.values
})

# Merge this with the original dataset to get product details
product_details = pd.merge(product_popularity, data_train[['product_id', 'product_name', 'type3']], on='product_id', how='left')

# Drop duplicates
product_details = product_details.drop_duplicates()

# Sort products based on their order count
product_details_sorted = product_details.sort_values(by='order_count', ascending=False)

# Considering constraints: max 100 refrigerated and max 100 frozen
refrigerated_products = product_details_sorted[product_details_sorted['type3'] == 'Refrigerated'].head(100)
frozen_products = product_details_sorted[product_details_sorted['type3'] == 'Frozen'].head(100)
other_products = product_details_sorted[~product_details_sorted['type3'].isin(['Refrigerated', 'Frozen'])].head(800)

# Combining the products to get top 1000 products for the channel
top_1000_products = pd.concat([refrigerated_products, frozen_products, other_products])

top_1000_products.head()


,product_id,order_count,product_name,type3
21689,21903,4689,Organic Baby Spinach,Refrigerated
42398,27966,2692,Organic Raspberries,Refrigerated
45090,27845,2584,Organic Whole Milk,Refrigerated
47674,22935,2175,Organic Yellow Onion,Refrigerated
49849,24964,2057,Organic Garlic,Refrigerated


In [25]:
# Check the number of products in each category within the top 1000 products
product_type_counts = top_1000_products['type3'].value_counts()

product_type_counts


Other           800
Refrigerated    100
Frozen          100
Name: type3, dtype: int64

In [26]:
# Displaying the top 10 products for each category: Other, Refrigerated, and Frozen

top_10_other = other_products.head(10)
top_10_refrigerated = refrigerated_products.head(10)
top_10_frozen = frozen_products.head(10)

top_10_other, top_10_refrigerated, top_10_frozen


(       product_id  order_count            product_name  type3
 0           24852         9153                  Banana  Other
 9153        13176         7395  Bag of Organic Bananas  Other
 16548       21137         5141    Organic Strawberries  Other
 26378       47209         4075    Organic Hass Avocado  Other
 30453       47766         3457         Organic Avocado  Other
 33910       47626         3001             Large Lemon  Other
 36911       16797         2768            Strawberries  Other
 39679       26209         2719                   Limes  Other
 57735        5876         1760           Organic Lemon  Other
 59495       28204         1757      Organic Fuji Apple  Other,
        product_id  order_count            product_name         type3
 21689       21903         4689    Organic Baby Spinach  Refrigerated
 42398       27966         2692     Organic Raspberries  Refrigerated
 45090       27845         2584      Organic Whole Milk  Refrigerated
 47674       22935        

In [27]:
import numpy as np
import pandas as pd
from tqdm import tqdm

# Load the dataset
data = pd.read_csv("train_dataset.csv")

# Define the initial state (based on our previous analysis)
initial_state = top_1000_products['product_id'].to_list()

# Define the cost function
def cost(solution):
    orders_with_solution = data[data['product_id'].isin(solution)]
    num_orders_with_solution = len(orders_with_solution['order_id'].unique())
    avg_percentage_items_from_solution = (len(orders_with_solution) / len(data)) * 100
    return -1 * (num_orders_with_solution + avg_percentage_items_from_solution)

# Define the neighbor function
def neighbor(solution):
    # Randomly replace one product from the solution with a product not in the solution
    product_out = np.random.choice(solution)
    product_in = np.random.choice(data[~data['product_id'].isin(solution)]['product_id'].unique())
    new_solution = [product_in if x==product_out else x for x in solution]
    return new_solution

# Simulated Annealing Algorithm
def simulated_annealing(initial_solution, T=40, T_min=0.00001, alpha=0.1, max_iterations=10000):
    current_solution = initial_solution
    current_cost = cost(current_solution)

    for i in tqdm(range(max_iterations)):
        next_solution = neighbor(current_solution)
        next_cost = cost(next_solution)

        delta_cost = next_cost - current_cost

        if delta_cost > 0 or np.random.uniform(0, 1) < np.exp(delta_cost / T):
            current_solution, current_cost = next_solution, next_cost

        T = T * alpha
        if T < T_min:
            break

    return current_solution

# Run the Simulated Annealing algorithm
optimized_solution = simulated_annealing(initial_state)

# Calculate the metrics for the optimized solution
orders_with_optimized_solution = data[data['product_id'].isin(optimized_solution)]
num_orders_with_optimized_solution = len(orders_with_optimized_solution['order_id'].unique())
avg_percentage_items_from_optimized_solution = (len(orders_with_optimized_solution) / len(data)) * 100

print(f"Number of orders using items from the channel: {num_orders_with_optimized_solution}")
print(f"Average % of items from the channel in each order: {avg_percentage_items_from_optimized_solution:.2f}%")


  0%|          | 6/10000 [00:00<14:12, 11.73it/s]

Number of orders using items from the channel: 56480
Average % of items from the channel in each order: 45.30%


optimized_solution：1000的list


Training set: 62,348 orders, 62,914 pieces of data  
Test set 15588 orders, 156491 pieces of data  
56k was used in the training set with an average utilization of 46%

# Tested with a test set and using the 5% alternatives rule

Results for optimized_solution

In [43]:
import pandas as pd
import numpy as np

# Load the test dataset
test_data = pd.read_csv("test_dataset.csv")

# Randomly select 5% of the orders from the test dataset
unique_orders = test_data['order_id'].unique()
selected_orders = np.random.choice(unique_orders, size=int(0 * len(unique_orders)), replace=False)

# Replace products in the selected orders with products from the optimized_solution (if they exist)
mask = test_data['order_id'].isin(selected_orders)
test_data.loc[mask, 'product_id'] = test_data.loc[mask, 'product_id'].apply(lambda x: x if x in optimized_solution else np.random.choice(optimized_solution))

# Calculate the metrics for the modified test dataset
orders_with_solution = test_data[test_data['product_id'].isin(optimized_solution)]
num_orders_with_solution = len(orders_with_solution['order_id'].unique())
avg_percentage_items_from_solution = (len(orders_with_solution) / len(test_data)) * 100

print(f"Number of orders using items from the channel: {num_orders_with_solution}")
print(f"Average % of items from the channel in each order: {avg_percentage_items_from_solution:.2f}%")


Number of orders using items from the channel: 14144
Average % of items from the channel in each order: 45.36%


Training set: 62,348 orders, 62,914 pieces of data  
Test set 15588 orders, 156491 pieces of data  
14,000 were used in the test set, with an average utilization rate of 49 percent

top_1000_products



In [29]:
top_1000_products_id = top_1000_products['product_id'].to_list()
top_1000_products_id

[21903,
 27966,
 27845,
 22935,
 24964,
 45007,
 39275,
 49683,
 4920,
 40706,
 30391,
 49235,
 42265,
 21616,
 17794,
 4605,
 44359,
 30489,
 27104,
 31717,
 27086,
 28985,
 46979,
 8518,
 41950,
 43352,
 22035,
 10749,
 34126,
 24184,
 48679,
 43961,
 35951,
 21938,
 34969,
 46667,
 12341,
 25890,
 5785,
 24838,
 28842,
 33731,
 8424,
 27521,
 44142,
 28199,
 18465,
 11520,
 29487,
 20114,
 9839,
 41220,
 34358,
 15290,
 3957,
 23909,
 47144,
 42736,
 4799,
 4210,
 33000,
 38689,
 30233,
 16759,
 34243,
 37687,
 40604,
 7781,
 33754,
 6184,
 19678,
 8193,
 26369,
 42828,
 42585,
 19348,
 21405,
 432,
 1463,
 16185,
 43295,
 32689,
 11777,
 14992,
 46676,
 31343,
 43789,
 39475,
 5212,
 38293,
 25659,
 5025,
 36011,
 42701,
 32655,
 10246,
 20842,
 2295,
 25146,
 31553,
 9076,
 20995,
 24489,
 17948,
 46802,
 37158,
 14678,
 42450,
 1158,
 49075,
 7963,
 28934,
 40545,
 16965,
 24561,
 32691,
 18918,
 32433,
 35887,
 20345,
 34584,
 2228,
 43504,
 9825,
 2452,
 311,
 12276,
 23236,
 2

In [51]:
import pandas as pd
import numpy as np

# Load the test dataset
test_data = pd.read_csv("test_dataset.csv")

# Randomly select 5% of the orders from the test dataset
unique_orders = test_data['order_id'].unique()
selected_orders = np.random.choice(unique_orders, size=int(0.05 * len(unique_orders)), replace=False)

# Replace products in the selected orders with products from the optimized_solution (if they exist)
mask = test_data['order_id'].isin(selected_orders)
test_data.loc[mask, 'product_id'] = test_data.loc[mask, 'product_id'].apply(lambda x: x if x in top_1000_products_id else np.random.choice(top_1000_products_id))

# Calculate the metrics for the modified test dataset
orders_with_solution = test_data[test_data['product_id'].isin(top_1000_products_id)]
num_orders_with_solution = len(orders_with_solution['order_id'].unique())
avg_percentage_items_from_solution = (len(orders_with_solution) / len(test_data)) * 100

print(f"Number of orders using items from the channel: {num_orders_with_solution}")
print(f"Average % of items from the channel in each order: {avg_percentage_items_from_solution:.2f}%")


Number of orders using items from the channel: 14229
Average % of items from the channel in each order: 48.70%


14212
Average % of items from the channel in each order: 48.55%

Training set: 62,348 orders, 62,914 pieces of data  
Test set 15588 orders, 156491 pieces of data  
14,000 were used in the test set, with an average utilization rate of 48 percent

In [45]:
import pandas as pd
import numpy as np

# Load the test dataset
test_data = pd.read_csv("train_dataset.csv")

# Randomly select 5% of the orders from the test dataset
unique_orders = test_data['order_id'].unique()
selected_orders = np.random.choice(unique_orders, size=int(0 * len(unique_orders)), replace=False)

# Replace products in the selected orders with products from the optimized_solution (if they exist)
mask = test_data['order_id'].isin(selected_orders)
test_data.loc[mask, 'product_id'] = test_data.loc[mask, 'product_id'].apply(lambda x: x if x in top_1000_products_id else np.random.choice(top_1000_products_id))

# Calculate the metrics for the modified test dataset
orders_with_solution = test_data[test_data['product_id'].isin(top_1000_products_id)]
num_orders_with_solution = len(orders_with_solution['order_id'].unique())
avg_percentage_items_from_solution = (len(orders_with_solution) / len(test_data)) * 100

print(f"Number of orders using items from the channel: {num_orders_with_solution}")
print(f"Average % of items from the channel in each order: {avg_percentage_items_from_solution:.2f}%")


Number of orders using items from the channel: 56550
Average % of items from the channel in each order: 45.89%
